In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LogisticRegression

# Load the new dataset

df = pd.read_csv('D600 Task 3 Dataset 1 Housing Information.csv')

# Drop the ID column and categorical features.
df = df.drop(columns=['ID',  'IsLuxury', 'Fireplace', 'HouseColor', 'Garage'])

# Preview the data
print("Data Shape:", df.shape)
df.head()

Data Shape: (7000, 17)


,Price,SquareFootage,NumBathrooms,NumBedrooms,BackyardSpace,CrimeRate,SchoolRating,AgeOfHome,DistanceToCityCenter,EmploymentRate,PropertyTaxRate,RenovationQuality,LocalAmenities,TransportAccess,Floors,Windows,PreviousSalePrice
0,255614.8992,566.62,1.000000,4,779.42,20.56,5.62,39.46,10.08,97.29,1.84,4.93,4.44,4.55,1,13,181861.54230
1,155586.0947,1472.34,1.000000,2,656.13,15.62,5.63,40.51,7.89,93.22,0.95,4.08,5.56,6.83,1,17,50042.59757
2,131050.8324,550.00,1.779354,3,754.57,12.47,9.20,48.38,23.74,96.60,1.87,4.26,8.07,8.48,2,34,48400.34440
3,151361.7125,941.81,2.035254,2,439.59,22.22,7.08,94.67,5.22,91.45,1.45,4.45,5.00,6.27,1,14,84594.12145
4,113167.6128,550.00,1.064644,3,353.03,8.28,5.93,16.80,43.13,86.50,1.26,3.36,5.46,6.99,1,21,22934.59654


In [5]:


# Remove rows with negative values
initial_rows = df.shape[0]
if 'PreviousSalePrice' in df.columns:
    df = df[df['PreviousSalePrice'] >= 0]
if 'Windows' in df.columns:
    df = df[df['Windows'] >= 0]
removed_rows = initial_rows - df.shape[0]
print(f"Removed {removed_rows} rows with negative values.")

# Check for Nulls
print("\nNull Values:\n", df.isnull().sum())

# Show cleaned dataframe info
print("\nCleaned Data Info:")
df.info()

Removed 78 rows with negative values.

Null Values:
 Price                   0
SquareFootage           0
NumBathrooms            0
NumBedrooms             0
BackyardSpace           0
CrimeRate               0
SchoolRating            0
AgeOfHome               0
DistanceToCityCenter    0
EmploymentRate          0
PropertyTaxRate         0
RenovationQuality       0
LocalAmenities          0
TransportAccess         0
Floors                  0
Windows                 0
PreviousSalePrice       0
dtype: int64

Cleaned Data Info:
<class 'pandas.core.frame.DataFrame'>
Index: 6922 entries, 0 to 6999
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Price                 6922 non-null   float64
 1   SquareFootage         6922 non-null   float64
 2   NumBathrooms          6922 non-null   float64
 3   NumBedrooms           6922 non-null   int64  
 4   BackyardSpace         6922 non-null   float64
 5   CrimeRat

In [7]:
# Display floats with 2 decimal places.
pd.set_option('display.float_format', lambda x: '%.2f' % x)
# Increase max columns to ensure all features are displayed
#pd.set_option('display.max_columns', None)

print("Descriptive Statistics for Price:\n")
display(df['Price'].describe())

initial_independent_features = df.drop(columns=['Price']).columns.tolist()

# Separate features into categorical and continuous lists.
categorical_independent_features = []
continuous_independent_features = []

for feature in initial_independent_features:
    if df[feature].dtype == 'category' or df[feature].dtype == 'bool':
        categorical_independent_features.append(feature)
    else:
        continuous_independent_features.append(feature)

# Display Descriptive Statistics for Continuous Variables.
print("\nDescriptive Statistics for Continuous Independent Variables:\n")
continuous_stats_list = []
for feature in continuous_independent_features:
    continuous_stats_list.append(df[feature].describe().rename(feature))
continuous_stats_df = pd.concat(continuous_stats_list, axis=1).T
display(continuous_stats_df)



Descriptive Statistics for Price:



count      6922.00
mean     308243.94
std      150002.90
min       85000.00
25%      193197.56
50%      280439.37
75%      392825.24
max     1046675.64
Name: Price, dtype: float64


Descriptive Statistics for Continuous Independent Variables:



,count,mean,std,min,25%,50%,75%,max
SquareFootage,6922.00,1050.99,425.84,550.00,663.71,1001.09,1344.42,2874.70
NumBathrooms,6922.00,2.13,0.95,1.00,1.29,2.00,2.77,5.81
NumBedrooms,6922.00,3.01,1.02,1.00,2.00,3.00,4.00,7.00
BackyardSpace,6922.00,511.23,280.00,0.39,300.71,495.81,703.90,1631.36
CrimeRate,6922.00,31.22,18.03,0.03,17.41,30.34,43.64,99.73
SchoolRating,6922.00,6.95,1.89,0.22,5.66,7.01,8.36,10.00
AgeOfHome,6922.00,46.86,31.82,0.01,20.77,42.69,67.31,178.68
DistanceToCityCenter,6922.00,17.44,11.99,0.00,7.81,15.60,25.17,65.20
EmploymentRate,6922.00,93.71,4.51,72.05,90.62,94.01,97.41,99.90
PropertyTaxRate,6922.00,1.50,0.50,0.01,1.16,1.49,1.84,3.36
